In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial 
import os
%matplotlib inline

In [2]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
GenreStats_DIR = "../gamedata/genreStats/"
PlatformStats_DIR = "../gamedata/platformStats/"
GenreScore_DIR = "../gamedata/genreScore/"
PlatformScore_DIR = "../gamedata/platformScore/"
WeightedScore_DIR = "../gamedata/weightedScore/"
TrainDataset_DIR = "../gamedata/train_dataset/"

In [3]:
dataset=pd.read_csv(DATA_DIR+DATA_FILE)#load original dataset

In [4]:
#Select a sub dataset where the genre equals to 'No genre' 
subdataset = dataset[dataset['genre']=='Poker']
#List of all events in 'No genre' subdataset
events = subdataset['eventname'].drop_duplicates()
#Count the amount of all event
#events_amount=events.count()
#List of all gameid  in 'No genre' subdataset
games = subdataset['gameid'].drop_duplicates()

In [5]:
events=list(set(events))
events.sort()


In [6]:
#get all gameid in this genre
def get_all_gameid(dataset):
    AllGameId = dataset['gameid']
    AllGameId = AllGameId.drop_duplicates()
    
    return AllGameId

In [7]:
#get all events in this game
def get_all_gameEvents(gameId,dataset):
    findGame = dataset[dataset['gameid'] == gameId]
    eventsInGame = findGame.drop_duplicates(['eventname'])
    eventsList = eventsInGame['eventname']
    
    return eventsList

In [8]:
allGameId = get_all_gameid(subdataset)
len(events)

271

In [9]:
def get_feature_vector(gameId,eventsVocabulary,dataset):
    ''' Map the presence of events to a vector
    :param gameId
    :type gameId: list of int
    :param eventsVocabulary 
    :type eventsVocabulary : list of str
    :param dataset
    :type dataset: dataframe
    :return feature_vect
    :type feature_vect: list of int
    '''
    eventsList = get_all_gameEvents(gameId,dataset)
    feature_vect = [0]*len(eventsVocabulary)
    
    for event in eventsList:
        if event in eventsVocabulary:
            idx = eventsVocabulary.index(event)
            print(idx)
            feature_vect[idx] = 1
    return feature_vect      

In [10]:
#get all feature vectors
train_dataset = [get_feature_vector(gameId,events,subdataset) for gameId in allGameId]

93
12
149
150
229
153
227
219
81
202
146
201
241
120
147
87
265
90
220
266
203
151
237
81
176
101
174
114
12
21
18
200
112
22
97
215
44
254
241
66
233
179
183
253
248
177
230
196
175
245
108
51
46
17
116
180
88
49
148
178
15
70
165
14
164
153
28
181
213
152
29
52
244
172
103
169
173
102
171
25
24
161
199
168
214
195
232
231
187
235
234
48
132
165
129
223
127
251
125
122
136
50
128
68
126
131
138
252
250
191
135
134
62
192
249
0
197
141
44
198
189
140
137
123
139
182
216
142
121
124
246
190
31
81
133
130
247
81
44
165
241
172
96
269
32
241
255
9
86
81
27
23
270
93
92
173
7
228
3
248
52
184
90
44
8
194
224
98
169
221
71
172
87
110
168
16
11
54
105
85
111
193
243
70
1
53
56
165
4
26
5
239
240
114
55
163
45
115
6
210
89
10
211
119
57
113
2
109
264
241
67
76
39
95
91
168
154
156
261
75
155
242
173
52
157
158
74
81
165
263
160
44
162
15
186
38
35
166
60
83
80
72
37
30
12
226
33
100
169
218
36
207
40
185
99
206
208
267
144
42
82
70
225
257
13
256
188
65
118
117
84
58
167
259
34
104
94
260
47


In [11]:
train_dataset = pd.DataFrame(train_dataset)
type(train_dataset)

pandas.core.frame.DataFrame

In [12]:
#train_dataset.to_csv(TrainDataset_DIR+"TrainNoGenre.csv")

In [13]:
train_dataset = np.array(train_dataset)
cond_prob_vect = (np.sum(train_dataset, axis=0))/(np.sum(train_dataset))


In [14]:
cond_prob_vect[:10]

array([ 0.00314465,  0.00314465,  0.00314465,  0.00314465,  0.00314465,
        0.00314465,  0.00314465,  0.00314465,  0.00314465,  0.00314465])

In [15]:
train_dataset = pd.DataFrame(train_dataset)
type(train_dataset)

pandas.core.frame.DataFrame

In [16]:
#amount of each event
num_events = np.sum(train_dataset,axis=0)

In [17]:
#Calculate P(ei|ej)
def calculateProb(i):
    eiej_tmp = pd.DataFrame()
    arr1=[]
    arr2=[]
    #amount = num 
    #for i in amount:
    arr1=[]
    for j in range(len(events)):
        eiej_tmp['i'] = train_dataset.iloc[:][i]
        eiej_tmp['j'] = train_dataset.iloc[:][j]
        eiej = eiej_tmp[(eiej_tmp['i']==1) & (eiej_tmp['j']==1)]
        p = len(eiej)/np.sum(eiej_tmp['i'])
        #print('p(e'+str(j)+'|e'+str(i)+ ')='+str(p) )
        arr1.append(p)
    #print(arr1)
    #arr2.append(arr1)
    print(i)
    return arr1

In [18]:
start = time.clock()
amount = range(len(events))
pool = ThreadPool(32) 
matrix = pool.map(calculateProb, amount)
pool.close()
pool.join()
end = time.clock()

#matrix = matrix = pd.DataFrame(calculateProb(train_dataset))

9
42
4812
66
903627847245306

33





57081391563351241821608769

75
7893


54













49
43
6710
7

345873


372885135231
91



707988
614
146

821916402264





5594



25



76
50
8
83
44
68625935


4111
32
749286143853229892371580
20

65











264795

1756




77
99
102
96
108
132129
126120117141123105






144111147174135162
168
138150

114
171



156183

165153
159


177
186
180

189
109
100
103
133
17597

118
124163

127
130169

145172
148142


139
184
112157106151136




121166
181

160
115
190
154
178187

101
110
104
98
176
143
167119131140164134125



128


107122173



182
152137158185


146
170149161

113
116


188
191
179
155
195
192
201198
207

216
219
222
204
240225210249228231




264237


246261
252
213255
234

243

258270

267
196
193
202
217
220
199
208
229
211
223205

232
250265241


247262238226



214235

253
259244

268
256
197
194
266
218
200
221203209

212
230224


251206

227
233
263
239
242
215
260
236
248
245
254
269257



In [19]:
#Time to create the probability matrix
print(end-start)
newmatrix = pd.DataFrame(matrix)

164.349107


In [285]:
newmatrix#.loc[:][93]

,0,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,269,270
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
2,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
3,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
4,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
5,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
6,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
7,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
8,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
9,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000


In [94]:
input_events = get_all_gameEvents(59,subdataset)
input_events

2308            handSummary
2309            achievement
2310       missionCompleted
2311       missionContinued
2312            tableSearch
2313         missionStarted
2314           tableConnect
2315             slotReward
2316            gameStarted
2318           rewardedItem
2319            messageSent
2320           rewardBundle
2321            transaction
2322            markerEvent
2324       missionAbandoned
2325               giftSent
2326      videoPokerGameEnd
2327             handAction
2329              slotStart
2332    videoPokerGameStart
2335         rewardedMetric
2336          missionFailed
Name: eventname, dtype: object

In [248]:
new_feature=['NaN']*len(events)
new_list1=[]
for data in input_events:
    if data in events:
        idx = events.index(data)
        new_feature[idx]=data
        #print(data,idx)
        find = newmatrix.loc[:][idx]
        findValue = find.idxmax()
       # print(find[findValue])
        if find[findValue]!=0:
            new_feature[findValue]=events[findValue]
            if events[findValue] not in new_list:
                new_list1.append(events[findValue])
            print(data,idx,'--------------',events[findValue],findValue)

handSummary 93 -------------- abandonTable 1
achievement 12 -------------- achievement 12
missionCompleted 149 -------------- markerEvent 120
missionContinued 150 -------------- markerEvent 120
tableSearch 229 -------------- markerEvent 120
missionStarted 153 -------------- applicationAction 14
tableConnect 227 -------------- markerEvent 120
slotReward 219 -------------- markerEvent 120
gameStarted 81 -------------- aI_Duel_end 0
rewardedItem 202 -------------- markerEvent 120
messageSent 146 -------------- markerEvent 120
rewardBundle 201 -------------- markerEvent 120
transaction 241 -------------- abandonTable 1
markerEvent 120 -------------- markerEvent 120
missionAbandoned 147 -------------- markerEvent 120
giftSent 87 -------------- abandonTable 1
videoPokerGameEnd 265 -------------- markerEvent 120
handAction 90 -------------- abandonTable 1
slotStart 220 -------------- markerEvent 120
videoPokerGameStart 266 -------------- markerEvent 120
rewardedMetric 203 -------------- marke

In [249]:
for data in input_events:
    if data not in input_events:
        new_list1.append(data)
new_list1

['handSummary',
 'achievement',
 'missionCompleted',
 'missionContinued',
 'tableSearch',
 'missionStarted',
 'tableConnect',
 'slotReward',
 'gameStarted',
 'rewardedItem',
 'messageSent',
 'rewardBundle',
 'transaction',
 'markerEvent',
 'missionAbandoned',
 'giftSent',
 'videoPokerGameEnd',
 'handAction',
 'slotStart',
 'videoPokerGameStart',
 'rewardedMetric',
 'missionFailed']

In [242]:
#Compare the new list and old list
new_old=pd.DataFrame()
new = pd.DataFrame(new_feature)
new = new.drop_duplicates()
new = new.reset_index()
del new['index']
# new_old['new']=new
# print(new_old['new'])


old=pd.DataFrame(input_events)
old = old.reset_index()
del old['index']
old
# new_old['old']=old
# print(new_old['old'])
# #new_drop = new_old[new_old['new']==0]
# # new_old=new_old.drop(new_drop.index)
# new_old = new_old.reset_index()
# del new_old['index']
# for data in new_old['new']:
#     event = new_old[new_old['old']==data]
#     print(data,'--------------',event['old'].values)



,eventname
0,handSummary
1,achievement
2,missionCompleted
3,missionContinued
4,tableSearch
5,missionStarted
6,tableConnect
7,slotReward
8,gameStarted
9,rewardedItem


In [245]:
# pred=[]
# for data in new_old['new']:
#     print(data)
#     if data in new_old['old']:
#         print(data)
#         pred.append(data)
# # new_1 = pd.DataFrame(new_list)
# input_1 = pd.DataFrame(input_events)
pred=new[~new.isin(old)] 
pred

,0
0,aI_Duel_end
1,abandonTable
2,NaN
3,achievement
4,applicationAction
5,gameStarted
6,giftSent
7,handAction
8,handSummary
9,markerEvent


In [250]:
list_1 = ['abandonTable','achievement', 'markerEvent','handSummary', 'missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [251]:
list_2 = ['achievement', 'markerEvent',,'handSummary', 'missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [253]:
list_3 = ['achievement', 'markerEvent','aI_Duel_end','handSummary', 'missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [262]:
list_4 = ['applicationAction','achievement', 'markerEvent','aI_Duel_end','handSummary', 'missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [265]:
list_5 = ['abandonTable','achievement', 'markerEvent','aI_Duel_end','handSummary','missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [273]:
list_6 = ['applicationAction','abandonTable','achievement', 'markerEvent','aI_Duel_end','handSummary','missionCompleted','missionContinued','tableSearch',
 'missionStarted', 'tableConnect','slotReward','gameStarted','rewardedItem'
 'messageSent', 'rewardBundle','transaction','missionAbandoned','giftSent', 'videoPokerGameEnd',
 'handAction','slotStart','videoPokerGameStart','rewardedMetric',
 'missionFailed']

In [274]:
allList=[]
allList.append(list_1)
allList.append(list_2)
allList.append(list_3)
allList.append(list_4)
allList.append(list_5)
allList.append(list_6)
allList1 = pd.DataFrame(allList)
allList1

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,abandonTable,achievement,markerEvent,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
1,achievement,markerEvent,applicationAction,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
2,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
3,applicationAction,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,...,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None
4,abandonTable,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,...,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None
5,applicationAction,abandonTable,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,...,rewardBundle,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed


In [275]:
for data in allList:
    new_feature_vector = get_newgame_feature(data,events)
    pred_probs = np.sum(cond_prob_vect*new_feature_vector)
    print(pred_probs)

0.122641509434
0.122641509434
0.122641509434
0.125786163522
0.125786163522
0.12893081761


In [175]:
new_old

,new,old
0,aI_Duel_end,handSummary
1,abandonTable,achievement
2,achievement,missionContinued
3,applicationAction,tableSearch
4,gameStarted,missionStarted
5,giftSent,tableConnect
6,handAction,slotReward
7,handSummary,gameStarted
8,markerEvent,rewardedItem
9,messageSent,messageSent


In [259]:
def get_newgame_feature(eventsList,eventsVocabulary):
    ''' Map the presence of events to a vector
    :param eventsList 
    :type list of events
    :param eventsVocabulary 
    :type eventsVocabulary : list of str
    :return feature_vect
    :type feature_vect: list of int
    '''
    feature_vect = [0]*len(eventsVocabulary)
    
    for event in eventsList:
        if event in eventsVocabulary:
            idx = eventsVocabulary.index(event)
#             print(idx)
            feature_vect[idx] = 1
    return feature_vect 

In [92]:
get_newgame_feature(input_events,events)

93
12
149
150
229
153
227
219
81
202
146
201
241
120
147
87
265
90
220
266
203
151


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [28]:
for data in events:
    
input_events[0]
#find0 = newmatrix.idxmax()
#find0

tournamentUnlocked


In [277]:
allList1

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,abandonTable,achievement,markerEvent,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
1,achievement,markerEvent,applicationAction,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
2,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,slotReward,...,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None,None
3,applicationAction,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,...,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None
4,abandonTable,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,tableConnect,...,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed,None
5,applicationAction,abandonTable,achievement,markerEvent,aI_Duel_end,handSummary,missionCompleted,missionContinued,tableSearch,missionStarted,...,rewardBundle,transaction,missionAbandoned,giftSent,videoPokerGameEnd,handAction,slotStart,videoPokerGameStart,rewardedMetric,missionFailed


In [278]:
allList1[0]

0         abandonTable
1          achievement
2          achievement
3    applicationAction
4         abandonTable
5    applicationAction
Name: 0, dtype: object

In [279]:
allList1[:][0]

0         abandonTable
1          achievement
2          achievement
3    applicationAction
4         abandonTable
5    applicationAction
Name: 0, dtype: object

In [281]:
allList1.loc[0][:]

0                abandonTable
1                 achievement
2                 markerEvent
3                 handSummary
4            missionCompleted
5            missionContinued
6                 tableSearch
7              missionStarted
8                tableConnect
9                  slotReward
10                gameStarted
11    rewardedItemmessageSent
12               rewardBundle
13                transaction
14           missionAbandoned
15                   giftSent
16          videoPokerGameEnd
17                 handAction
18                  slotStart
19        videoPokerGameStart
20             rewardedMetric
21              missionFailed
22                       None
23                       None
Name: 0, dtype: object

In [284]:
newmatrix.loc[12][:]

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     1.000000
13     0.333333
14     0.333333
15     0.666667
16     0.000000
17     0.333333
18     0.333333
19     0.000000
20     0.000000
21     0.333333
22     0.333333
23     0.000000
24     0.333333
25     0.333333
26     0.000000
27     0.000000
28     0.333333
29     0.333333
         ...   
241    1.000000
242    0.333333
243    0.000000
244    0.333333
245    0.333333
246    0.000000
247    0.000000
248    0.333333
249    0.000000
250    0.000000
251    0.000000
252    0.000000
253    0.333333
254    0.333333
255    0.000000
256    0.333333
257    0.333333
258    0.333333
259    0.333333
260    0.333333
261    0.333333
262    0.333333
263    0.333333
264    0.333333
265    0.333333
266    0.333333
267    0.333333
268    0.333333
269    0.000000
270    0.000000
Name: 12, Length: 271, d